In [3]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
#import lightgbm as lgb

import random
import os
import re

# import optuna
# from optuna import Trial, visualization
# from optuna.samplers import TPESampler

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error

warnings.filterwarnings(action='ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 지점 결측치 채우기

In [73]:
path = '/content/drive/MyDrive/TEST_AWS/'
file_list = os.listdir(path)
file_list

['옹도.csv',
 '세종전의.csv',
 '세천.csv',
 '세종금남.csv',
 '오월드.csv',
 '세종연서.csv',
 '세종고운.csv',
 '양화.csv',
 '안도.csv',
 '예산.csv',
 '아산.csv',
 '성거.csv',
 '계룡.csv',
 '대산.csv',
 '북격렬비도.csv',
 '공주.csv',
 '당진.csv',
 '논산.csv',
 '홍성죽도.csv',
 '서천.csv',
 '대천항.csv',
 '홍북.csv',
 '춘장대.csv',
 '청양.csv',
 '호도.csv',
 '장동.csv',
 '유구.csv',
 '태안.csv',
 '정산.csv',
 '정안.csv']

In [7]:
for file in file_list:
  f = pd.read_csv(path + file)
  f['지점'].fillna(f['지점'][0], inplace = True)
  f.to_csv(path + file, index = False)


In [8]:
sample = pd.read_csv(path + '공주.csv')
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728 entries, 0 to 7727
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  7728 non-null   int64  
 1   연도          7728 non-null   int64  
 2   지점          7728 non-null   object 
 3   기온(°C)      3120 non-null   float64
 4   풍향(deg)     3120 non-null   float64
 5   풍속(m/s)     3120 non-null   float64
 6   강수량(mm)     3120 non-null   float64
 7   습도(%)       3120 non-null   float64
 8   지역번호        7728 non-null   int64  
 9   월           7728 non-null   int64  
 10  일           7728 non-null   int64  
 11  시           7728 non-null   int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 724.6+ KB


# 2. 날짜, 시간 전처리 

In [9]:
# # 일시 컬럼 이용해 월, 일, 시, 분 단위로 컬럼 split
# def date_split(df):
#     df['날짜'] = df.일시.str.split().str[0]
#     df['시간'] = df.일시.str.split().str[1]
#     df['월'] = df.날짜.str.split('-').str[0]
#     df['일'] = df.날짜.str.split('-').str[1]
#     df['시'] = df.시간.str.split(':').str[0]
    
#     # 1시간마다 측정했으므로 분 단위 필요 X
#     # df['분'] = df.시간.str.split(':').str[1]
    
#     # 분할했으니 기존 컬럼 제거
#     df.drop(['날짜', '시간', '일시'],axis = 1, inplace=True)
    
#     # Split 진행한 컬럼들 형변환
#     df['월'] = df['월'].astype('int64')
#     df['일'] = df['일'].astype('int64')
#     df['시'] = df['시'].astype('int64')
#     return df

# for file in file_list:
#   f = pd.read_csv(path + file)
#   f = date_split(f)
#   f.to_csv(path + file, index = False)

AttributeError: ignored

In [10]:
sample = pd.read_csv(path + '공주.csv')
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728 entries, 0 to 7727
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  7728 non-null   int64  
 1   연도          7728 non-null   int64  
 2   지점          7728 non-null   object 
 3   기온(°C)      3120 non-null   float64
 4   풍향(deg)     3120 non-null   float64
 5   풍속(m/s)     3120 non-null   float64
 6   강수량(mm)     3120 non-null   float64
 7   습도(%)       3120 non-null   float64
 8   지역번호        7728 non-null   int64  
 9   월           7728 non-null   int64  
 10  일           7728 non-null   int64  
 11  시           7728 non-null   int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 724.6+ KB


## TEST SET 병합

- Matching해서 하나씩 하면 Data가 적어 연관성 파악을 잘못할 것 같기 때문에, 모델 성능을 위해 하나로 병합 후 진행

In [74]:
for idx in range(len(file_list)):
  if idx == 0:
    first = pd.read_csv(path + file_list[0])
  elif idx == 1:
    second = pd.read_csv(path + file_list[1])
    test = pd.concat([first, second])
  else:
    tmp_df = pd.read_csv(path + file_list[idx])
    test = pd.concat([test, tmp_df])

test.shape  
  

(231840, 12)

In [75]:
test.drop(['Unnamed: 0'], axis = 1, inplace = True)
test

,연도,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),지역번호,월,일,시
0,4,옹도,0.325434,0.071944,0.248082,0.0,0.680,3,1,1,0
1,4,옹도,0.322275,0.070833,0.253197,0.0,0.667,3,1,1,1
2,4,옹도,0.320695,0.098056,0.237852,0.0,0.668,3,1,1,2
3,4,옹도,0.320695,0.055833,0.276215,0.0,0.658,3,1,1,3
4,4,옹도,0.323855,0.040556,0.230179,0.0,0.671,3,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...
7723,4,정안,0.429700,0.819167,0.099744,0.0,0.363,3,11,18,19
7724,4,정안,0.413902,0.788333,0.092072,0.0,0.525,3,11,18,20
7725,4,정안,0.398104,0.789167,0.102302,0.0,0.495,3,11,18,21
7726,4,정안,0.390205,0.810556,0.094629,0.0,0.600,3,11,18,22


In [76]:
test = test.reset_index(drop=True)
test.to_csv('test.csv', index = False)

In [79]:
len(test['강수량(mm)'][test['강수량(mm)'] != 0])

144372

# 3. 월, 일, 시, 지점, 지역번호 -> 강수량

## SEED 고정

In [80]:
SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

## TRAIN,TEST DATASET LOAD

In [110]:
train_path = '/content/drive/MyDrive/'

train = pd.read_csv(train_path+'train_first.csv')
train_X = train[['월', '일', '시', '지점', '지역번호']]
train_Y = train['강수량(mm)']
train.head()

,연도,측정소,PM2.5,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),지역번호,...,일,시,fix_월,fix_일,fix_시,fix_기온(°C),fix_습도(%),fix_강수량(mm),fix_풍향(deg),fix_풍속(m/s)
0,0,아름동,0.044,세종고운,0.170616,0.053333,0.017903,0.0,0.764,1,...,1,0,3,1,2,0,2,0,1.0,2.0
1,0,아름동,0.048,세종고운,0.167457,0.099444,0.020460,0.0,0.759,1,...,1,1,3,1,2,0,2,0,1.0,2.0
2,0,아름동,0.060,세종고운,0.162717,0.134167,0.012788,0.0,0.760,1,...,1,2,3,1,1,0,2,0,2.0,2.0
3,0,아름동,0.064,세종고운,0.157978,0.000000,0.005115,0.0,0.747,1,...,1,3,3,1,1,0,2,0,1.0,1.0
4,0,아름동,0.064,세종고운,0.148499,0.000000,0.007673,0.0,0.791,1,...,1,4,3,1,1,0,2,0,1.0,1.0


In [111]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580546 entries, 0 to 580545
Data columns (total 21 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   연도           580546 non-null  int64  
 1   측정소          580546 non-null  object 
 2   PM2.5        580546 non-null  float64
 3   지점           580546 non-null  object 
 4   기온(°C)       580546 non-null  float64
 5   풍향(deg)      580546 non-null  float64
 6   풍속(m/s)      580546 non-null  float64
 7   강수량(mm)      580546 non-null  float64
 8   습도(%)        580546 non-null  float64
 9   지역번호         580546 non-null  int64  
 10  월            580546 non-null  int64  
 11  일            580546 non-null  int64  
 12  시            580546 non-null  int64  
 13  fix_월        580546 non-null  int64  
 14  fix_일        580546 non-null  int64  
 15  fix_시        580546 non-null  int64  
 16  fix_기온(°C)   580546 non-null  int64  
 17  fix_습도(%)    580546 non-null  int64  
 18  fix_강수량(mm)  580546 non-

In [112]:
test = pd.read_csv('/content/test.csv')
test_X = test[['월', '일', '시', '지점', '지역번호']]
test_Y = test['강수량(mm)']
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   연도       231840 non-null  int64  
 1   지점       231840 non-null  object 
 2   기온(°C)   93600 non-null   float64
 3   풍향(deg)  93600 non-null   float64
 4   풍속(m/s)  93600 non-null   float64
 5   강수량(mm)  93600 non-null   float64
 6   습도(%)    93600 non-null   float64
 7   지역번호     231840 non-null  int64  
 8   월        231840 non-null  int64  
 9   일        231840 non-null  int64  
 10  시        231840 non-null  int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 19.5+ MB


In [113]:
# 결측치 처리
test['기온(°C)'].fillna(0, inplace =True)
test['강수량(mm)'].fillna(0, inplace =True)
test['습도(%)'].fillna(0, inplace =True)
test['풍향(deg)'].fillna(0, inplace =True)
test['풍속(m/s)'].fillna(0, inplace =True)

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   연도       231840 non-null  int64  
 1   지점       231840 non-null  object 
 2   기온(°C)   231840 non-null  float64
 3   풍향(deg)  231840 non-null  float64
 4   풍속(m/s)  231840 non-null  float64
 5   강수량(mm)  231840 non-null  float64
 6   습도(%)    231840 non-null  float64
 7   지역번호     231840 non-null  int64  
 8   월        231840 non-null  int64  
 9   일        231840 non-null  int64  
 10  시        231840 non-null  int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 19.5+ MB


## DATA 전처리

In [114]:
train_columns = train_X.columns
for column in train_columns:
    train_X[column] = train_X[column].astype('str')
train_Y = train_Y.astype(float)

In [115]:
test_columns = test_X.columns
for column in test_columns:
    test_X[column] = test_X[column].astype('str')
test_Y = test_Y.astype(float)

## Modeling

In [116]:
import copy

params_cat = {
    "task_type" : "GPU",
    "devices" : '0',
    "random_state": SEED,
    'learning_rate': 0.04574578205475402, 
    'bagging_temperature': 0.12172958098369972, 
    'n_estimators': 2000, 
    'max_depth': 8, 
    'random_strength': 28, 
    'l2_leaf_reg': 1.6285455533915874e-05, 
    'min_child_samples': 18, 
    'max_bin': 441, 
    'od_type': 'Iter',
    "cat_features" : list(train_X.columns),
}

model = CatBoostRegressor(**params_cat)
model.fit(
    train_X,
    train_Y,
    #early_stopping_rounds=10,
    verbose=False,
)

pred = model.predict(test_X)


In [117]:
idx = test[test['강수량(mm)'] == 0.0].index
len(idx)

225708

In [118]:
len(pred[idx])

225708

In [119]:
# Index 뽑아서 기존의 결측치만 채우기
test.loc[list(idx),'강수량(mm)'] = pred[idx]

In [120]:
idx

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            231830, 231831, 231832, 231833, 231834, 231835, 231836, 231837,
            231838, 231839],
           dtype='int64', length=225708)

In [121]:
print(test.loc[idx[1], '강수량(mm)'])
print(pred[idx][1])

-0.0005380115649479683
-0.0005380115649479683


In [122]:
test['강수량(mm)'][test['강수량(mm)'] < 0] = 0
test[test['강수량(mm)'] >= 0].shape

(231840, 11)

In [123]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   연도       231840 non-null  int64  
 1   지점       231840 non-null  object 
 2   기온(°C)   231840 non-null  float64
 3   풍향(deg)  231840 non-null  float64
 4   풍속(m/s)  231840 non-null  float64
 5   강수량(mm)  231840 non-null  float64
 6   습도(%)    231840 non-null  float64
 7   지역번호     231840 non-null  int64  
 8   월        231840 non-null  int64  
 9   일        231840 non-null  int64  
 10  시        231840 non-null  int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 19.5+ MB


In [124]:
pred

array([-0.00075963, -0.00053801, -0.0005065 , ...,  0.00076805,
       -0.00074842, -0.00139187])

# 4. 월, 일, 시, 지점, 강수량, 습도

In [125]:
train_X = train[['월', '일', '시', '지점', '지역번호', '강수량(mm)']]
train_Y = train['습도(%)']
train.head()

,연도,측정소,PM2.5,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),지역번호,...,일,시,fix_월,fix_일,fix_시,fix_기온(°C),fix_습도(%),fix_강수량(mm),fix_풍향(deg),fix_풍속(m/s)
0,0,아름동,0.044,세종고운,0.170616,0.053333,0.017903,0.0,0.764,1,...,1,0,3,1,2,0,2,0,1.0,2.0
1,0,아름동,0.048,세종고운,0.167457,0.099444,0.020460,0.0,0.759,1,...,1,1,3,1,2,0,2,0,1.0,2.0
2,0,아름동,0.060,세종고운,0.162717,0.134167,0.012788,0.0,0.760,1,...,1,2,3,1,1,0,2,0,2.0,2.0
3,0,아름동,0.064,세종고운,0.157978,0.000000,0.005115,0.0,0.747,1,...,1,3,3,1,1,0,2,0,1.0,1.0
4,0,아름동,0.064,세종고운,0.148499,0.000000,0.007673,0.0,0.791,1,...,1,4,3,1,1,0,2,0,1.0,1.0


In [126]:
test_X = test[['월', '일', '시', '지점', '지역번호', '강수량(mm)']]
test_Y = test['습도(%)']
test_X.head()

,월,일,시,지점,지역번호,강수량(mm)
0,1,1,0,옹도,3,0.0
1,1,1,1,옹도,3,0.0
2,1,1,2,옹도,3,0.0
3,1,1,3,옹도,3,0.0
4,1,1,4,옹도,3,0.0


In [127]:
train_columns = train_X.columns
for column in train_columns:
    train_X[column] = train_X[column].astype('str')
train_Y = train_Y.astype(float)

In [128]:
test_columns = test_X.columns
for column in test_columns:
    test_X[column] = test_X[column].astype('str')
test_Y = test_Y.astype(float)

In [129]:
params_cat = {
    "task_type" : "GPU",
    "devices" : '0',
    "random_state": SEED,
    'learning_rate': 0.04574578205475402, 
    'bagging_temperature': 0.12172958098369972, 
    'n_estimators': 2000, 
    'max_depth': 8, 
    'random_strength': 28, 
    'l2_leaf_reg': 1.6285455533915874e-05, 
    'min_child_samples': 18, 
    'max_bin': 441, 
    'od_type': 'Iter',
    "cat_features" : list(train_X.columns),
}

model = CatBoostRegressor(**params_cat)
model.fit(
    train_X,
    train_Y,
    #early_stopping_rounds=10,
    verbose=False,
)

pred = model.predict(test_X)

In [130]:
idx = test[test['습도(%)'] == 0.0].index
test.loc[list(idx),'습도(%)'] = pred[idx]

In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   연도       231840 non-null  int64  
 1   지점       231840 non-null  object 
 2   기온(°C)   231840 non-null  float64
 3   풍향(deg)  231840 non-null  float64
 4   풍속(m/s)  231840 non-null  float64
 5   강수량(mm)  231840 non-null  float64
 6   습도(%)    231840 non-null  float64
 7   지역번호     231840 non-null  int64  
 8   월        231840 non-null  int64  
 9   일        231840 non-null  int64  
 10  시        231840 non-null  int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 19.5+ MB


# 5. 월, 일, 시, 지점, 강수량, 습도, 풍향

In [132]:
train_X = train[['월', '일', '시', '지점', '지역번호', '강수량(mm)','습도(%)']]
train_Y = train['풍향(deg)']

test_X = test[['월', '일', '시', '지점', '지역번호', '강수량(mm)','습도(%)']]
test_Y = test['풍향(deg)']

In [133]:
train_columns = train_X.columns
for column in train_columns:
    train_X[column] = train_X[column].astype('str')
train_Y = train_Y.astype(float)

test_columns = test_X.columns
for column in test_columns:
    test_X[column] = test_X[column].astype('str')
test_Y = test_Y.astype(float)

In [134]:
model = CatBoostRegressor(**params_cat)
model.fit(
    train_X,
    train_Y,
    #early_stopping_rounds=10,
    verbose=False,
)

pred = model.predict(test_X)

In [135]:
idx = test[test['풍향(deg)'] == 0.0].index
test.loc[list(idx),'풍향(deg)'] = pred[idx]

# 6. 월, 일, 시, 지점, 강수량, 습도, 풍향, 풍속

In [136]:
train_X = train[['월', '일', '시', '지점', '지역번호', '강수량(mm)','습도(%)', '풍향(deg)']]
train_Y = train['풍속(m/s)']

test_X = test[['월', '일', '시', '지점', '지역번호', '강수량(mm)','습도(%)', '풍향(deg)']]
test_Y = test['풍속(m/s)']

In [137]:
train_columns = train_X.columns
for column in train_columns:
    train_X[column] = train_X[column].astype('str')
train_Y = train_Y.astype(float)

test_columns = test_X.columns
for column in test_columns:
    test_X[column] = test_X[column].astype('str')
test_Y = test_Y.astype(float)

In [138]:
model = CatBoostRegressor(**params_cat)
model.fit(
    train_X,
    train_Y,
    #early_stopping_rounds=10,
    verbose=False,
)

pred = model.predict(test_X)

In [139]:
idx = test[test['풍속(m/s)'] == 0.0].index
test.loc[list(idx),'풍속(m/s))'] = pred[idx]

# 7. 월, 일, 시, 지점, 강수량, 습도, 풍향, 풍속, 기온

In [141]:
train_X = train[['월', '일', '시', '지점', '지역번호', '강수량(mm)','습도(%)', '풍향(deg)', '풍속(m/s)']]
train_Y = train['기온(°C)']

test_X = test[['월', '일', '시', '지점', '지역번호', '강수량(mm)','습도(%)', '풍향(deg)', '풍속(m/s)']]
test_Y = test['기온(°C)']

In [142]:
train_columns = train_X.columns
for column in train_columns:
    train_X[column] = train_X[column].astype('str')
train_Y = train_Y.astype(float)

test_columns = test_X.columns
for column in test_columns:
    test_X[column] = test_X[column].astype('str')
test_Y = test_Y.astype(float)

In [143]:
model = CatBoostRegressor(**params_cat)
model.fit(
    train_X,
    train_Y,
    #early_stopping_rounds=10,
    verbose=False,
)

pred = model.predict(test_X)

In [144]:
idx = test[test['기온(°C)'] == 0.0].index
test.loc[list(idx),'기온(°C)'] = pred[idx]

# DATA -> CSV

In [146]:
def temper(df):
    df[ "fix_기온(°C)"] = 0
    df.loc[(0.277 <= df["기온(°C)"]) & (df['기온(°C)'] < 0.491), "fix_기온(°C)"] = 1
    df.loc[(0.491 <= df["기온(°C)"]) & (df['기온(°C)'] < 0.691), 'fix_기온(°C)'] = 2
    return df
test = temper(test)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   연도          231840 non-null  int64  
 1   지점          231840 non-null  object 
 2   기온(°C)      231840 non-null  float64
 3   풍향(deg)     231840 non-null  float64
 4   풍속(m/s)     231840 non-null  float64
 5   강수량(mm)     231840 non-null  float64
 6   습도(%)       231840 non-null  float64
 7   지역번호        231840 non-null  int64  
 8   월           231840 non-null  int64  
 9   일           231840 non-null  int64  
 10  시           231840 non-null  int64  
 11  풍속(m/s))    143008 non-null  float64
 12  fix_기온(°C)  231840 non-null  int64  
dtypes: float64(6), int64(6), object(1)
memory usage: 23.0+ MB


In [149]:
def humid(df):
    df["fix_습도(%)"]= 0
    df.loc[df['습도(%)'] < 0.273, "fix_습도(%)"] = 1
    df.loc[(0.501 <= df["습도(%)"]) & (df['습도(%)'] < 0.978), "fix_습도(%)"] = 2
    df.loc[0.987 <= df["습도(%)"], "fix_습도(%)"] = 3
    return df
test = humid(test)

In [150]:
def rain(df):
    df["fix_강수량(mm)"] = 0
    df.loc[df['강수량(mm)'] > 0, "fix_강수량(mm)"] = 1
    return df
test = rain(test)

In [152]:
def preprocess_speed_df_(df_):
    df_['fix_풍속(m/s)'] = df_['풍속(m/s)'].copy()
    df_['fix_풍속(m/s)'][df_['풍속(m/s)'] < 0.0125] = 1
    df_['fix_풍속(m/s)'][(df_['풍속(m/s)'] < 0.025) & (df_['풍속(m/s)'] >= 0.0125)] = 2
    df_['fix_풍속(m/s)'][(df_['풍속(m/s)'] < 0.035) & (df_['풍속(m/s)'] >= 0.025)] = 3
    df_['fix_풍속(m/s)'][(df_['풍속(m/s)'] < 0.05) & (df_['풍속(m/s)'] >= 0.035)] = 4
    df_['fix_풍속(m/s)'][(df_['풍속(m/s)'] < 0.1) & (df_['풍속(m/s)'] >= 0.05)] = 5
    df_['fix_풍속(m/s)'][(df_['풍속(m/s)'] < 0.2) & (df_['풍속(m/s)'] >= 0.1)] = 6
    df_['fix_풍속(m/s)'][df_['풍속(m/s)'] >= 0.2] = 7
    df_['fix_풍속(m/s)'] = df_['fix_풍속(m/s)'].astype('str')
    return df_

preprocess_speed_df_(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   연도           231840 non-null  int64  
 1   지점           231840 non-null  object 
 2   기온(°C)       231840 non-null  float64
 3   풍향(deg)      231840 non-null  float64
 4   풍속(m/s)      231840 non-null  float64
 5   강수량(mm)      231840 non-null  float64
 6   습도(%)        231840 non-null  float64
 7   지역번호         231840 non-null  int64  
 8   월            231840 non-null  int64  
 9   일            231840 non-null  int64  
 10  시            231840 non-null  int64  
 11  fix_기온(°C)   231840 non-null  int64  
 12  fix_습도(%)    231840 non-null  int64  
 13  fix_강수량(mm)  231840 non-null  int64  
 14  fix_풍속(m/s)  231840 non-null  object 
dtypes: float64(5), int64(8), object(2)
memory usage: 26.5+ MB


In [153]:
def preprocess_direction_df_(df_):
    df_['fix_풍향(deg)'] = df_['풍향(deg)'].copy()
    df_['fix_풍향(deg)'][df_['풍향(deg)'] < 0.125] = 1
    df_['fix_풍향(deg)'][(df_['풍향(deg)'] < 0.25) & (df_['풍향(deg)'] >= 0.125)] = 2
    df_['fix_풍향(deg)'][(df_['풍향(deg)'] < 0.375) & (df_['풍향(deg)'] >= 0.25)] = 3
    df_['fix_풍향(deg)'][(df_['풍향(deg)'] < 0.5) & (df_['풍향(deg)'] >= 0.375)] = 4
    df_['fix_풍향(deg)'][(df_['풍향(deg)'] < 0.625) & (df_['풍향(deg)'] >= 0.5)] = 5
    df_['fix_풍향(deg)'][(df_['풍향(deg)'] < 0.75) & (df_['풍향(deg)'] >= 0.625)] = 6
    df_['fix_풍향(deg)'][(df_['풍향(deg)'] < 0.875) & (df_['풍향(deg)'] >= 0.75)] = 7
    df_['fix_풍향(deg)'][df_['풍향(deg)'] >= 0.875] = 8
    df_['fix_풍향(deg)'] = df_['fix_풍향(deg)'].astype('str')
    return df_

preprocess_direction_df_(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   연도           231840 non-null  int64  
 1   지점           231840 non-null  object 
 2   기온(°C)       231840 non-null  float64
 3   풍향(deg)      231840 non-null  float64
 4   풍속(m/s)      231840 non-null  float64
 5   강수량(mm)      231840 non-null  float64
 6   습도(%)        231840 non-null  float64
 7   지역번호         231840 non-null  int64  
 8   월            231840 non-null  int64  
 9   일            231840 non-null  int64  
 10  시            231840 non-null  int64  
 11  fix_기온(°C)   231840 non-null  int64  
 12  fix_습도(%)    231840 non-null  int64  
 13  fix_강수량(mm)  231840 non-null  int64  
 14  fix_풍속(m/s)  231840 non-null  object 
 15  fix_풍향(deg)  231840 non-null  object 
dtypes: float64(5), int64(8), object(3)
memory usage: 28.3+ MB


In [155]:
def date_preprocessing(df):
    # 월 기준 
    df['fix_월'] = df['월'].apply(lambda x: 1 if x in [6, 7, 8, 9] else (2 if x in [3, 4, 5, 10] else 3))
    # 일 기준
    df['fix_일'] = df['일'].map(lambda x: 1 if (5 <= x <= 6) or (11 <= x <= 24) or (x == 1) or (x >= 28) else 2)
    # 시간대별 기준 
    df['fix_시'] = df['시'].map(lambda x: 1 if (2 <= x <= 5) or (9 <= x <= 16) else 2)
    return df
test = date_preprocessing(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   연도           231840 non-null  int64  
 1   지점           231840 non-null  object 
 2   기온(°C)       231840 non-null  float64
 3   풍향(deg)      231840 non-null  float64
 4   풍속(m/s)      231840 non-null  float64
 5   강수량(mm)      231840 non-null  float64
 6   습도(%)        231840 non-null  float64
 7   지역번호         231840 non-null  int64  
 8   월            231840 non-null  int64  
 9   일            231840 non-null  int64  
 10  시            231840 non-null  int64  
 11  fix_기온(°C)   231840 non-null  int64  
 12  fix_습도(%)    231840 non-null  int64  
 13  fix_강수량(mm)  231840 non-null  int64  
 14  fix_풍속(m/s)  231840 non-null  object 
 15  fix_풍향(deg)  231840 non-null  object 
 16  fix_월        231840 non-null  int64  
 17  fix_일        231840 non-null  int64  
 18  fix_시        231840 non-

In [157]:
test.to_csv('/content/drive/MyDrive/preprocessed_test.csv', index = False)

In [159]:
train['측정소'].nunique()

17